In [1]:
import pandas as pd
import os
import sys
import json
import requests
import gc
from tqdm import tqdm
from datetime import datetime, timedelta
import datetime as dt
import openpyxl
import re
import datetime as dt

import warnings
warnings.filterwarnings(action='ignore')

import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 폰트 경로 직접 지정
font_path = 'C:/Windows/Fonts/malgun.ttf'  # 맑은 고딕의 경로
font_prop = fm.FontProperties(fname=font_path, size=12)

# 전역 폰트 설정
plt.rcParams['font.family'] ='Malgun Gothic'  # 맑은 고딕으로 설정
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

In [13]:
keys = pd.read_csv('../../cp_data.csv')

In [52]:
key = 1

In [55]:
media = keys['media'][key]
access_key = keys['id'][key]
secret_key = keys['secret'][key]


In [82]:
start_date = "20240516"
end_date = "20240531"

In [83]:
# report_type = 'orders'
report_type = 'cancels'

In [84]:
import requests 
import hmac
import hashlib
from time import gmtime, strftime
from datetime import datetime, timedelta
import pandas as pd
import time

REQUEST_METHOD = "GET"
DOMAIN = "https://api-gateway.coupang.com"
URL = f"/v2/providers/affiliate_open_api/apis/openapi/v1/reports/{report_type}"

def generateHmac(method, url, secretKey, accessKey, queryString=""):
    datetimeGMT = strftime('%y%m%d', gmtime()) + 'T' + strftime('%H%M%S', gmtime()) + 'Z'
    message = datetimeGMT + method + url + queryString
    signature = hmac.new(bytes(secretKey, 'utf-8'), message.encode('utf-8'), hashlib.sha256).hexdigest()
    return f"CEA algorithm=HmacSHA256, access-key={accessKey}, signed-date={datetimeGMT}, signature={signature}"

from tqdm import tqdm  # Import tqdm

def fetch_data(startDate, endDate, page, retries=3):
    queryString = f"startDate={startDate}&endDate={endDate}&page={page}"
    fullURL = f"{DOMAIN}{URL}?{queryString}"
    authorization = generateHmac(REQUEST_METHOD, URL, SECRET_KEY, ACCESS_KEY, queryString)
    headers = {"Authorization": authorization, "Content-Type": "application/json"}
    
    for attempt in range(retries):
        try:
            response = requests.get(fullURL, headers=headers, timeout=60)
            
            if response.status_code == 200:
                return response.json()
            else:
                print(f"Failed to fetch data: HTTP {response.status_code} - {response.text}")
                time.sleep(2)  # Wait before retrying
                
        except requests.exceptions.RequestException as e:
            print(f"Request failed: {e}")
            time.sleep(2)  # Wait before retrying
    
    return None

def fetch_data_in_chunks(start_date, end_date):
    start = datetime.strptime(start_date, '%Y%m%d')
    end = datetime.strptime(end_date, '%Y%m%d')
    results = []

    date_range = (end - start).days // 30 + 1  # Calculate the number of chunks
    date_chunks = tqdm(range(date_range), desc="Fetching date ranges")

    for _ in date_chunks:
        chunk_end = start + timedelta(days=29)
        if chunk_end > end:
            chunk_end = end

        page = 0
        while True:
            response = fetch_data(start.strftime('%Y%m%d'), chunk_end.strftime('%Y%m%d'), page)
            if response:
                data = response.get('data', [])
                if data:
                    results.extend(data)
                    if len(data) < 1000:
                        break
                    page += 1
                else:
                    break
            else:
                break

        start = chunk_end + timedelta(days=1)
        if start > end:
            break

    return results

# Define your access keys and date range
ACCESS_KEY = access_key
SECRET_KEY = secret_key



# Collect all data
all_data = fetch_data_in_chunks(start_date, end_date)
df = pd.DataFrame(all_data)

df['click_time'] = df['subParam'].apply(lambda x: x.split('_')[1] if isinstance(x, str) and '_' in x else None)
df['subParam'] = df['subParam'].apply(lambda x: x.split('_')[0] if isinstance(x, str) and '_' in x else None)

df['click_time'] = pd.to_datetime(df['click_time'])
df['click_time'] = df['click_time'].dt.strftime("%Y-%m-%d %H:%M")

Fetching date ranges:   0%|          | 0/1 [00:00<?, ?it/s]

Failed to fetch data: HTTP 504 - {
  "code" : "ERROR",
  "message" : "Request timed out, if the situation continues consider applying timeout extension.",
  "transactionId" : "6e2c8809-eb11-4f4e-b835-bce721f0d9f4"
}


Fetching date ranges:   0%|          | 0/1 [00:10<?, ?it/s]


In [85]:
df.groupby('date')['gmv'].sum()

date
20240516   -6007145.0
20240517   -6172214.0
20240518   -4261222.0
20240519   -8499844.0
20240520   -6726893.0
20240521   -4370944.0
20240522   -5256474.0
20240523   -4939330.0
20240524   -3596470.0
20240525   -3303132.0
20240526   -4224330.0
20240527   -5326099.0
20240528   -5335943.0
20240529   -4668248.0
20240530   -5650852.0
20240531   -5129118.0
Name: gmv, dtype: float64

In [86]:
if media == "cpDynamic":
    if report_type == "orders" :
        df.to_parquet(f'../result/cp/dynamic/orders/{media}_{report_type}_{start_date}_{end_date}.parquet')
    else :
        df.to_parquet(f'../result/cp/dynamic/cancels/{media}_{report_type}_{start_date}_{end_date}.parquet')
else:
    if report_type == "orders" :
        df.to_parquet(f'../result/cp/reco/orders/{media}_{report_type}_{start_date}_{end_date}.parquet')
    else :
        df.to_parquet(f'../result/cp/reco/cancels/{media}_{report_type}_{start_date}_{end_date}.parquet')